In [55]:
import polars as pl
import builtins

In [56]:
# import plotly.io as pio
# pio.renderers.default = "png" 

In [57]:
import plotly.express as px

In [58]:
path = r"C:\Users\Rudra\Desktop\bihar-viz\data\BL05.parquet"

In [59]:
fdf = pl.read_parquet(path)

fdf.head(2)

Survey_Name,Year,FSU_Serial_No,Sector,State,NSS_Region,District,Stratum,Sub_stratum,Panel,Sub_sample,FOD_Sub_Region,Sample_SU_No,Sample_Sub_Division_No,Second_Stage_Stratum_No,Sample_Household_No,Questionnaire_No,Level,Item_Code,OutOfHome_Consumption_Quantity,OutOfHome_Consumption_Value,Total_Consumption_Quantity,Total_Consumption_Value,Source,Multiplier,Sector_label,State_label,ingest_timestamp,ingest_date
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,bool,i64,i64,str,str,f64,i64,i64,i64,str,str,str,str
"""HCES""",2024,27865,2,10,101,14,12,2,5,1,1010,1,null,3,15,false,5,141,null,null,1.0,84,1,30655,"""urban""","""Bihar""","""2026-01-07 05:26:18.239+00""","""2026-01-07"""
"""HCES""",2024,27865,2,10,101,14,12,2,5,1,1010,1,null,3,15,false,5,143,null,null,1.5,180,1,30655,"""urban""","""Bihar""","""2026-01-07 05:26:18.239+00""","""2026-01-07"""


In [60]:
fdf = fdf[[
 'FSU_Serial_No','Sector','NSS_Region','District','Stratum',
 'Sub_stratum','Panel','Sub_sample','FOD_Sub_Region',
 'Sample_SU_No','Sample_Sub_Division_No','Second_Stage_Stratum_No',
 'Sample_Household_No','Questionnaire_No','Item_Code',
 'OutOfHome_Consumption_Quantity','OutOfHome_Consumption_Value',
 'Total_Consumption_Quantity','Total_Consumption_Value'
 ]]

In [61]:
fdf = fdf.with_columns(
    (pl.col("OutOfHome_Consumption_Quantity").cast(pl.Float64, strict=False)),
    (pl.col("OutOfHome_Consumption_Value").cast(pl.Float64, strict=False))
)

In [62]:
category_mapping = {
    129: "cereals",
    139: "cereal substitute",
    159: "pulses & products",
    179: "salt & sugar",
    169: "milk & milk products",
    219: "vegetables",
    239: "fruits (fresh)",
    249: "fruits (dry)",
    199: "egg, fish & meat",
    189: "edible oil",
    269: "spices",
    279: "beverages",
    289: "served processed food",
    299: "packaged processed food"
}

In [63]:
category_mapping = {
    61: "rice-free",
    62: "wheat/atta-free",
    70: "coarse grains-free",
    101: "rice – PDS",
    102: "rice – other sources",
    103: "chira",
    105: "muri",
    106: "other rice products (khoi/lawa, etc.)",
    107: "wheat/atta – PDS",
    108: "wheat/atta – other sources",
    110: "maida",
    111: "suji/rawa",
    112: "vermicelli (sewai)",
    114: "other wheat products",
    1: "coarse grains – PDS",
    2: "coarse grains – other sources",
    122: "other cereals & products",
    129: "cereals: sub-total"
}

In [64]:
map_df = pl.DataFrame({
    "Item_Code": builtins.list(category_mapping.keys()),
    "category_mapped": builtins.list(category_mapping.values()),
})

fdf = fdf.join(map_df, on="Item_Code", how="left")

In [65]:
cat_df = fdf.filter(
    pl.col("category_mapped").is_not_null()
    ) \
    [
    'OutOfHome_Consumption_Quantity',
    'OutOfHome_Consumption_Value',
    'Total_Consumption_Quantity',
    'Total_Consumption_Value',
    "category_mapped"
    ] \
     .group_by("category_mapped").agg(
        pl.col("OutOfHome_Consumption_Value").sum().alias("out_home_value"),
        pl.col("OutOfHome_Consumption_Quantity").sum().alias("out_home_qty"),
        pl.col("Total_Consumption_Quantity").sum().alias("total_qty"),
        pl.col("Total_Consumption_Value").sum().alias("total_value")
     ).with_columns(
         out_of_home_avg_pice = pl.when(pl.col("out_home_qty") > 0) 
                        .then(pl.col("out_home_value") / pl.col("out_home_qty"))
                        .otherwise(0),
         total_avg_pice = pl.when(pl.col("total_qty") > 0) 
                        .then(pl.col("total_value") / pl.col("total_qty"))
                        .otherwise(0),
     ) 

In [66]:
cat_df

category_mapped,out_home_value,out_home_qty,total_qty,total_value,out_of_home_avg_pice,total_avg_pice
str,f64,f64,f64,i64,f64,f64
"""suji/rawa""",0.0,0.0,5294.788,235353,0.0,44.449938
"""cereals: sub-total""",4.140426e6,132868.3,888887.082,21450998,31.16188,24.132422
"""muri""",0.0,0.0,7025.588,401067,0.0,57.08661
"""coarse grains – other sources""",13112.0,607.5,2026.9,49226,21.583539,24.286349
"""other rice products (khoi/lawa…",0.0,0.0,284.43,11424,0.0,40.16454
…,…,…,…,…,…,…
"""vermicelli (sewai)""",0.0,0.0,1629.262,177745,0.0,109.095406
"""other wheat products""",0.0,0.0,156.4,5415,0.0,34.622762
"""other cereals & products""",470.0,10.0,165.55,7181,47.0,43.376623


In [67]:
cat_df.columns

['category_mapped',
 'out_home_value',
 'out_home_qty',
 'total_qty',
 'total_value',
 'out_of_home_avg_pice',
 'total_avg_pice']

In [68]:
fig = px.bar(
    cat_df,
    x="category_mapped",
    y="out_home_value"
)

fig.show()

In [69]:
fig = px.scatter(
    cat_df,
    x="total_qty",
    y="total_value",
    color="category_mapped",
    symbol="category_mapped",
    size="total_value"
)

fig.show()

In [70]:
fig = px.violin(
    cat_df,
    x="total_qty",
)

fig.show()

In [71]:
fig = px.bar(
    cat_df,
    x="total_qty",
    y="category_mapped",
    orientation="h",
    title="Total Consumption Quantity by Category"
)

fig.show()


In [72]:
fig = px.box(
    cat_df,
    y="total_value",
    title="Distribution of Total Consumption Value"
)

fig.show()


In [79]:
price_df = cat_df.select([
    "category_mapped",
    "out_of_home_avg_pice",
    "total_avg_pice"
]).unpivot(
    index="category_mapped",
    on=["out_of_home_avg_pice", "total_avg_pice"],
    variable_name="price_type",
    value_name="price"
)

fig = px.box(
    price_df,
    x="price_type",
    y="price",
    title="Price Comparison: Out-of-Home vs Total"
)

fig.show()


In [80]:
fig = px.scatter(
    cat_df,
    x="total_qty",
    y="total_value",
    color="category_mapped",
    title="Total Quantity vs Total Value by Category",
    labels={
        "total_qty": "Total Quantity",
        "total_value": "Total Value"
    }
)

fig.show()


In [81]:
fig = px.scatter(
    cat_df,
    x="total_qty",
    y="total_avg_pice",
    color="category_mapped",
    title="Total Quantity vs Average Price"
)

fig.show()


In [82]:
qty_df = cat_df.select([
    "category_mapped",
    "out_home_qty",
    "total_qty"
]).unpivot(
    index="category_mapped",
    on=["out_home_qty", "total_qty"],
    variable_name="quantity_type",
    value_name="quantity"
)

fig = px.bar(
    qty_df,
    x="category_mapped",
    y="quantity",
    color="quantity_type",
    barmode="group",
    title="Out-of-Home vs Total Quantity by Category"
)

fig.show()


In [83]:
val_df = cat_df.select([
    "category_mapped",
    "out_home_value",
    "total_value"
]).unpivot(
    index="category_mapped",
    on=["out_home_value", "total_value"],
    variable_name="value_type",
    value_name="value"
)

fig = px.bar(
    val_df,
    x="category_mapped",
    y="value",
    color="value_type",
    barmode="stack",
    title="Out-of-Home vs Total Consumption Value"
)

fig.show()


In [ ]:
fig = px.scatter(
    cat_df,
    x="total_qty",
    y="total_value",
    size="total_avg_pice",
    color="category_mapped",
    title="Consumption Pattern by Category (Bubble Chart)",
    size_max=40
)

fig.show()